In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as soup
from multiprocessing import Pool
import time

In [ ]:
def getIds(page):
    print page
    url = "https://baelige.buenosaires.gob.ar/budgets/1/investments?order=confidence_score&page={0}".format(page)
    r = requests.get(url, timeout=10)
    if r.status_code == 200:
        page_soup = soup(r.text,"html.parser")
        propuestas = page_soup.findAll("div",{"class":"budget-investment-content"})
        apoyo = page_soup.findAll("span",{"class":"total-supports"})
        if propuestas is not None:
            ids = [ p.find('h3').a['href'].split('/')[-1] for p in propuestas ]
        if apoyo is not None:
            apoyos = [ p.text.strip().split(" ")[0] for p in apoyo]
        time.sleep(3)
        return {"apoyos":apoyos, "id":ids}

# 2128

p = Pool(10)
records = p.map(getIds, range(1, 2128))

if len(records) > 0:
    propuestasDf = pd.DataFrame ({
        "id": [y for x in records for y in x["id"]],
        "apoyos": [y for x in records for y in x["apoyos"]]
    })
    propuestasDf["apoyos"] = propuestasDf["apoyos"].replace(to_replace="Sin", value=0).astype(int)
    propuestasDf.to_csv('baelige-ids-apoyos.csv', index=False, encoding='utf-8')

In [2]:
propuestasDf = pd.read_csv('baelige-ids-apoyos.csv')

In [3]:
def getPropuesta(page):
    url = "https://baelige.buenosaires.gob.ar/budgets/1/investments/{0}".format(page[0])
    r = requests.get(url, timeout=10)
    if r.status_code == 200:
        page_soup = soup(r.text,"html.parser")
        
        column = page_soup.select(".budget-investment-show .column")[0]

        titulo = column.select('h1')[0].text
        descri = column.select('blockquote')[0].text
        info = column.select('.budget-investment-info')[0].contents
        comuna = info[8].strip()
        fecha = info[6].strip()
        tag = column.find("ul",{"class":"tags"}).li.a.text
        pid = column.find("p",{"id":"investment_code"}).strong.text

        content = ""
        f = False
        for e in column.children:
            if f:
                if e.name == "p":
                    content += e.text
            if e.name == "ul":
                f = True

        comentarios = page_soup.select(".js-comments-count")[0].text.replace("(", "").replace(")", "")
        
        time.sleep(3)
        
        return {
            "titulo": titulo,
            "descri": descri,
            "content": content,
            "tag": tag,
            "comuna": comuna,
            "fecha": fecha,
            "comentarios": comentarios,
            "apoyos": page[1],
            "pid": pid
        }

# # 2128

p = Pool(10)
pdf = propuestasDf
records = p.map(getPropuesta, zip(pdf["id"], pdf["apoyos"]))

if len(records) > 0:
    pDf = pd.DataFrame ({
        "titulo": [ x["titulo"] for x in records],
        "resumen": [ x["descri"] for x in records],
        "content": [ x["content"] for x in records],
        "tag": [ x["tag"] for x in records],
        "comuna": [ x["comuna"] for x in records],
        "fecha": [ x["fecha"] for x in records],
        "comentarios": [ x["comentarios"] for x in records],
        "apoyos": [ x["apoyos"] for x in records],
        "pid": [ x["pid"] for x in records]
    })
    pDf.to_csv('baelige-propuestas.csv', index=False, encoding='utf-8')

ConnectionError: None: Max retries exceeded with url: /budgets/1/investments/6789 (Caused by None)